In [ ]:
#The task will take ~6 hours so this is needed to prevent your cluster from pausing

# This code snippet allows for modification of the autopause behavior for the current notebook server. This is
# a stop-gap approach until we have integration in the Workbench/Notebooks UI to change the autopause settings
# for notebook servers. *Warning*: an unpaused server incurs much higher cost and may exhaust free credits rapidly.
# This setting is not entirely sticky. In production, it will be lost after 1-2w, depending on activity and when
# the server was created.# To disable autopause, set it to false
# To change the autopause threshold for how long to wait after last UI activity before
# autopausing (unit: minutes), set autopauseThreshold (default is 30m)
!curl -X PATCH \
  --header 'Content-Type: application/json' \
  --header "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
  -d '{ "autopause": false, "autopauseThreshold": 60 }' \
  "https://notebooks.firecloud.org/api/cluster/${GOOGLE_PROJECT}/${CLUSTER_NAME}" \
  | jq '.'

In [3]:
%%bash
gsutil cp 'gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.dict' .
gsutil cp 'gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.fasta' .
gsutil cp 'gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.fasta.fai' .

Copying gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.dict...
Copying gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.fasta...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/Homo_sapiens_assembly19.fasta.fai...
Copying gs://fc-secure-36ff2d2c-861e-4a9f-b04a-6f08bb178679/data/extract/probe_info.csv...
\ [4 files][  3.0 GiB/  3.0 GiB]   83.1 MiB/s                                   
Operation completed over 4 objects/3.0 GiB.                                      


In [ ]:
%%bash
PROJECT_ID=fc-aou-cdr-synth-test
DATASET_NAME=microarray_data
TABLE_NAME=probe_info

OUTPUT_VCF="aou.vcf"
REF="Homo_sapiens_assembly19.fasta"

EXTRACT_PROBE_CLAUSE="--probe-info-csv probe_info.csv"

gatk ArrayExtractCohort -R "${REF}" \
   -O $OUTPUT_VCF \
   --use-compressed-data "true" \
   --cohort-extract-table $PROJECT_ID.$DATASET_NAME.arrays_003 \
   --local-sort-max-records-in-ram 10000000 \
   --sample-info-table $PROJECT_ID.$DATASET_NAME.sample_list \
   $EXTRACT_PROBE_CLAUSE \
   --project-id $PROJECT_ID

In [ ]:
%%bash
bgzip -c aou.vcf > aou.vcf.gz

In [18]:
import argparse
import subprocess
import sys
import time

# Given the 'expected' cohort vcf and the actual cohort VCF, we want to check four things:
# 1) The VCFs have the same number of variants
# 2) The VCFs have the same number of samples
# 3) The VCFs have the same allele frequency across sites (with fuzziness for float variance)
# 4) All genotypes match for, say, the first sample in the cohort
start = time.time()
#parser = argparse.ArgumentParser(description="Validate a VCF generated by cohort extraction")
#parser.add_argument('--expected', required=True, help="A bgzipped VCF. An index file of the same basename must exist in the same directory.")
#parser.add_argument('--actual', required=True, help="A bgzipped VCF. An index file of the same basename must exist in the same directory.")
#parser.add_argument('--sample', help="The name of one sample for which to validate all genotypes in both VCFs")
#args = parser.parse_args()

bcf_bin = "./bcftools/bcftools"
expected_vcf = "Merged.vcf.gz"
actual_vcf = "aou.vcf.gz"
sample_name = None

# bcftools stats counts a bunch of stuff in each vcf - records, types of variants,
# types of substitution, distribution of variants, etc. We don't care about most
# of them but also can't filter down. This handles 1) and 2).
expected_stats_proc = subprocess.run([bcf_bin, "stats", expected_vcf], capture_output=True, text=True)
actual_stats_proc = subprocess.run([bcf_bin, "stats", actual_vcf], capture_output=True, text=True)
expected_stats = expected_stats_proc.stdout.split("\n")
actual_stats = actual_stats_proc.stdout.split("\n")

# SN stands for Summary Numbers, which covers things like number of samples,
# number of records, number of indels, SNPs, multiallelic sites, etc. May as
# well check all of them.
expected_summary = [stat for stat in expected_stats if stat.startswith('SN')]
actual_summary = [stat for stat in actual_stats if stat.startswith('SN')]
# We'll grab number of samples for later convenience
samples = 0
for i in range(0, len(expected_summary)):
    expected_stat = expected_summary[i].split("\t")
    actual_stat = actual_summary[i].split("\t")
    if (expected_stat[-1] != actual_stat[-1]):
      print(f"Discrepancy in summary stat '{expected_stat[-2]}': Expected {expected_stat[-1]}, Actual {actual_stat[-2]}")
      sys.exit()
    if (actual_stat[2] == "number of samples:"):
      samples = expected_stat[3]

# grab the first sample and pull that sample out of both vcfs.
sample_to_check_gts = sample_name if sample_name else None
if (sample_to_check_gts is None):
    expected_sample_names_proc = subprocess.run(["bcftools", "query", "-l", expected_vcf], capture_output=True, text=True)
    expected_sample_names = expected_sample_names_proc.stdout.replace("\'", "").split("\n")
    sample_to_check_gts = expected_sample_names[0]

# bcftools query extracts particular fields from each field, such as allele frequency or genotype.
expected_query_proc = subprocess.run([bcf_bin, "query", "-f", "'%CHROM %POS %AF [%GT]\n'", "-s", sample_to_check_gts, expected_vcf], capture_output=True, text=True)
actual_query_proc = subprocess.run([bcf_bin, "query", "-f", "'%CHROM %POS %AF [%GT]\n'", "-s", sample_to_check_gts, actual_vcf], capture_output=True, text=True)
# apPARently there is a newline on the end thus the splice
expected_query_stringy = expected_query_proc.stdout.replace("\'", "").split("\n")[:-1]
actual_query_stringy = actual_query_proc.stdout.replace("\'", "").split("\n")[:-1]
expected_af = [af.split() for af in expected_query_stringy]
actual_af = [af.split() for af in actual_query_stringy]

# munge - map by chromosome then by position.
expected_by_chrom_pos = {}
for chrom, pos, af, gt in expected_af:
    # af is pretty straightforward.
    if chrom in expected_by_chrom_pos:
      if pos in expected_by_chrom_pos:
        expected_by_chrom_pos[chrom][pos]['af'] = af
        expected_by_chrom_pos[chrom][pos]['gt'] = gt
      else:
        expected_by_chrom_pos[chrom][pos] = {}
        expected_by_chrom_pos[chrom][pos]['af'] = af
        expected_by_chrom_pos[chrom][pos]['gt'] = gt
    else:
      expected_by_chrom_pos[chrom] = {}
      expected_by_chrom_pos[chrom][pos] = {}
      expected_by_chrom_pos[chrom][pos]['af'] = af
      expected_by_chrom_pos[chrom][pos]['gt'] = gt

# and the same for actual.
actual_by_chrom_pos = {}
for chrom, pos, af, gt in actual_af:
    # actual af
    if chrom in actual_by_chrom_pos:
      if pos in actual_by_chrom_pos:
        actual_by_chrom_pos[chrom][pos]['af'] = af
        actual_by_chrom_pos[chrom][pos]['gt'] = gt
      else:
        actual_by_chrom_pos[chrom][pos] = {}
        actual_by_chrom_pos[chrom][pos]['af'] = af
        actual_by_chrom_pos[chrom][pos]['gt'] = gt
    else:
      actual_by_chrom_pos[chrom] = {}
      actual_by_chrom_pos[chrom][pos] = {}
      actual_by_chrom_pos[chrom][pos]['af'] = af
      actual_by_chrom_pos[chrom][pos]['gt'] = gt

# chromosomes SHOULD be the same for both of these. we're probably doing them all on humans.
for chrom in expected_by_chrom_pos.keys():
    expected_positions = sorted(expected_by_chrom_pos[chrom].keys())
    actual_positions = sorted(actual_by_chrom_pos[chrom].keys())
    if expected_positions != actual_positions:
      print(f"Discrepancy on chromosome {chrom}: sequenced positions are not the same")
      sys.exit()
    else:
      for pos in expected_positions:
        # validate AF. This handles 3)
        if expected_by_chrom_pos[chrom][pos]['af'] != actual_by_chrom_pos[chrom][pos]['af']:
          print(f"Discrepancy in allele frequency on chromosome {chrom} at position {pos}: Expected {expected_by_chrom_pos[chrom][pos]['af']}, Actual {actual_af_by_chrom_pos[chrom][pos]['af']}")
          sys.exit()
        # validate gt. This handles 4)
        if expected_by_chrom_pos[chrom][pos]['gt'] != actual_by_chrom_pos[chrom][pos]['gt']:
          print(f"Discrepancy in genotype on chromosome {chrom} at position {pos}: Expected {expected_by_chrom_pos[chrom][pos]['gt']}, Actual {actual_by_chrom_pos[chrom][pos]['gt']}")
          sys.exit()

print("All good, this looks like the same set of people.")
end = time.time()
print(f"Elapsed time: {end-start} seconds for {samples} samples")

Discrepancy in summary stat 'number of records:': Expected 1909964, Actual number of records:


SystemExit: 

In [19]:
%%bash
./bcftools/bcftools stats aou.vcf.gz

# This file was produced by bcftools stats (1.10.2-107-gd17d9cd+htslib-1.10.2-117-ga79009b) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  aou.vcf.gz
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	aou.vcf.gz
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types w